In [7]:
import pandas as pd
import numpy as np
from scipy import stats
import re


bills = pd.read_csv("bills.csv")
calls = pd.read_csv("calls.csv")
services = pd.read_csv("services.csv")
customers = pd.read_csv("customers.csv")


def remove_duplicates(df):
    return df.drop_duplicates()


def standardize_categorical_case(df):
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.strip().str.lower()
    return df


def clean_phone_numbers(df, col_name):
    df[col_name] = df[col_name].astype(str).apply(lambda x: re.sub(r'\D', '', x))
    return df

def impute_missing_data(df):
    for col in df.columns:
        if df[col].dtype in ['float64', 'int64']:
            if df[col].isnull().sum() > 0:
                df[col] = df[col].fillna(df[col].median())
        elif df[col].dtype == 'object':
            if df[col].isnull().sum() > 0:
                df[col] = df[col].fillna(df[col].mode()[0])
    return df


def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

def convert_data_types():
    if 'date_of_birth' in customers.columns:
        customers['date_of_birth'] = pd.to_datetime(customers['date_of_birth'], errors='coerce')
    if 'call_start' in calls.columns and 'call_end' in calls.columns:
        calls['call_start'] = pd.to_datetime(calls['call_start'], errors='coerce')
        calls['call_end'] = pd.to_datetime(calls['call_end'], errors='coerce')
    if 'bill_amount' in bills.columns:
        bills['bill_amount'] = pd.to_numeric(bills['bill_amount'], errors='coerce')
    if 'call_duration_minutes' in calls.columns:
        calls['call_duration_minutes'] = pd.to_numeric(calls['call_duration_minutes'], errors='coerce')
    if 'data_usage_mb' in services.columns:
        services['data_usage_mb'] = pd.to_numeric(services['data_usage_mb'], errors='coerce')


def validate_relations():
    valid_customer_ids = set(customers['customer_id'])
    if 'customer_id' in calls.columns:
        calls.drop(calls[~calls['customer_id'].isin(valid_customer_ids)].index, inplace=True)
    if 'customer_id' in bills.columns:
        bills.drop(bills[~bills['customer_id'].isin(valid_customer_ids)].index, inplace=True)
    if 'customer_id' in services.columns:
        services.drop(services[~services['customer_id'].isin(valid_customer_ids)].index, inplace=True)


bills = remove_duplicates(bills)
calls = remove_duplicates(calls)
services = remove_duplicates(services)
customers = remove_duplicates(customers)


bills = standardize_categorical_case(bills)
calls = standardize_categorical_case(calls)
services = standardize_categorical_case(services)
customers = standardize_categorical_case(customers)


if 'phone_number' in customers.columns:
    customers = clean_phone_numbers(customers, 'phone_number')


convert_data_types()


bills = impute_missing_data(bills)
calls = impute_missing_data(calls)
services = impute_missing_data(services)
customers = impute_missing_data(customers)


if 'call_duration_minutes' in calls.columns:
    calls = remove_outliers(calls, 'call_duration_minutes')
if 'data_usage_mb' in services.columns:
    services = remove_outliers(services, 'data_usage_mb')
if 'bill_amount' in bills.columns:
    bills = remove_outliers(bills, 'bill_amount')

validate_relations()


bills.to_csv("bills_cleaned.csv", index=False)
calls.to_csv("calls_cleaned.csv", index=False)
services.to_csv("services_cleaned.csv", index=False)
customers.to_csv("customers_cleaned.csv", index=False)




In [9]:
pip install sqlalchemy pymysql


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from sqlalchemy import create_engine

# Load cleaned CSVs
customers = pd.read_csv("customers_cleaned.csv")
bills = pd.read_csv("bills_cleaned.csv")
calls = pd.read_csv("calls_cleaned.csv")
services = pd.read_csv("services_cleaned.csv")

# Database connection info — update with your actual credentials
username = 'root'
password = 'admin'
host = 'localhost'
port = 3306
database = 'telecom_data'

# Create connection string for SQLAlchemy
connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

# Upload each DataFrame
customers.to_sql('customers', con=engine, if_exists='append', index=False)
bills.to_sql('bills', con=engine, if_exists='append', index=False)
calls.to_sql('calls', con=engine, if_exists='append', index=False)
services.to_sql('services', con=engine, if_exists='append', index=False)

print("✅ All data inserted into MySQL successfully!")


✅ All data inserted into MySQL successfully!
